In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install fastparquet
!sudo apt-get install libsnappy-dev
!python3 -m pip install  python-snappy
!python3 -m pip install pyarrow
!pip install tensorflow-addons
!pip install --upgrade numpy

     |████████████████████████████████| 1.2 MB 12.6 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 1.5 MB 39.2 MB/s 
     |████████████████████████████████| 125 kB 71.1 MB/s 
  Created wheel for thrift: filename=thrift-0.15.0-cp37-cp37m-linux_x86_64.whl size=348185 sha256=862ce4c7d539b9b81654a3c1a44523776431c54d985a6381e42394f1cfa83c82
  Stored in directory: /root/.cache/pip/wheels/ba/1f/8e/e6fd36837eecf3d1f2b23f1729477e8e06558d8d60b7093f51
Successfully built thrift
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libsnappy-dev
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 27.2 kB of archives.
After this operation, 108 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsnappy-dev amd64 1.1.7-1 [27.2 kB]
Fetched 27.2 kB in 0s (520 kB/s)
debconf: unable to init

In [3]:
import os
os.chdir('/content/gdrive/My Drive/phyto_curves_reco')

import re
import pandas as pd
from pred_functions import predict
from keras.models import load_model
from time import time

from tensorflow_addons.optimizers import RectifiedAdam, Lookahead


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
# Model and nomenclature loading
cnn = load_model('/content/gdrive/My Drive/Models/SSLAMM/cnn_small')

In [5]:
os.chdir('/content/gdrive/My Drive/data/SSLAMM/')

In [23]:
tn = pd.read_csv('L2/Pulse/train_test_nomenclature.csv')
tn.columns = ['name', 'id']

# Define where to look the data at and where to store preds
export_folder = "full_data/"
export_files = os.listdir(export_folder)

pulse_regex = "_Pulse" 
files_to_pred = [file for file in export_files if re.search(pulse_regex, file) and re.search('.parq', file)] # The files containing the data to predict

In [24]:
files_to_pred.sort()
files_to_pred = files_to_pred[1500: 1550]

In [25]:
# Create a log file in the destination folder: list of the already predicted files
preds_store_folder = "/content/gdrive/My Drive/Results/SSLAMM/Preds1"  # Where to store the predictions
log_path = preds_store_folder + "/pred_logs.txt" # Register where write the already predicted files

if not(os.path.isfile(log_path)):
    open(log_path, 'w+').close()

In [26]:
start_time = time()
nb_files_to_pred = len(files_to_pred)
print(nb_files_to_pred)

50


In [27]:
for idx, file in enumerate(files_to_pred):
    print('Currently predicting ' + file + ' ' + str(idx))
    path = export_folder + '/' + file
    is_already_pred = False
    
    # Check if file has already been predicted.
    with open(log_path, "r") as log_file:
        if file in log_file.read(): 
            is_already_pred = True
    
    if not(is_already_pred): # If not, perform the prediction
        # Predict the values
        predict(path, preds_store_folder,  cnn, tn,\
            is_ground_truth = False)

        # Write in the logs that this file is already predicted
        with open(log_path, "a") as log_file:
            log_file.write(file + '\n')

        step_time = time()
        average_pred_time = (step_time - start_time) / (idx + 1)
        remaining_time = average_pred_time * (nb_files_to_pred - idx - 1)
        print('Average per file pred time', average_pred_time, idx, 'files already predicted')
        print('Remaining time before end of pred', remaining_time)
          
    else:
        print(file, 'already predicted')

Currently predicting Labelled_Pulse6_2019-11-29 21h59.parq 0
Labelled_Pulse6_2019-11-29 21h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-29 23h59.parq 1
Labelled_Pulse6_2019-11-29 23h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 01h59.parq 2
Labelled_Pulse6_2019-11-30 01h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 03h59.parq 3
Labelled_Pulse6_2019-11-30 03h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 05h59.parq 4
Labelled_Pulse6_2019-11-30 05h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 07h59.parq 5
Labelled_Pulse6_2019-11-30 07h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 09h59.parq 6
Labelled_Pulse6_2019-11-30 09h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 11h59.parq 7
Labelled_Pulse6_2019-11-30 11h59.parq already predicted
Currently predicting Labelled_Pulse6_2019-11-30 13h59.parq 8
Lab